In [1]:
# input 
data_directory = ".//hooklogs//morstar/" # make sure the last character is '/'
in_tag = "morstar"
in_parseFirstPar = True # keep first parameter or not
in_window = 1

In [2]:
# 1.Get a dict of feature profiles (out: a hkName list, a featureProfile dict)
%run FeatureHooklog3.ipynb
Hooklog = FeatureHooklog3

hkName_li = list(filter(lambda f:f.endswith('.trace.hooklog'), os.listdir(data_directory))) # hooklog Name List

# ps: get Hkli without timestamp
fp_dict = {f:Hooklog(data_directory + f, 0).getHkli_noContainTS() for f in hkName_li } # Feature Profile dict (of hkName_li)

In [3]:
# 2.Get a dict of pairwise alignment result 
# (in: a featureProfile dict; out: alignmentProfile dict, alingmnt baseline)

%run Alignment3.ipynb
BASE = hkName_li[0] # randomly pick a BASE trace log
fpBASE = fp_dict[BASE]
align_dict = {hk:pairwise_NW( fpBASE, fp_dict[hk], 2, -1, -3, 1, 0, 1)[2] for hk in hkName_li } # pairwise all

In [4]:
# 3. Get matchMatrix, gapSeqList
# (in: alignmentProfile dict; out: matchMatrix, gapSeqList)

%run StructMatchGap3.ipynb
data = structMatchGap(align_dict)
matchMatrix = data[0]
gapSeqList = data[1]

In [5]:
# 4. Get stageMatrix 
# (in: matchMatrix, gapSeqList; out: stageMatrix)

% run StageMatrix.ipynb
stageMatrixResult = stageMatrix(matchMatrix, gapSeqList)

In [6]:
#  5. Get Motif Elements
# (in: stageMatrixResult, BASE; out: Motif object)

% run Motif.ipynb
Motif = Motif(stageMatrixResult, BASE)

total segment: 10
No Gap.
